# 1. Adat előkészítése

In [2]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
print("Setup Complete")
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

Setup Complete


In [3]:
def classificate(value):
    target='neutral'
    if(value>=0.001 and value<=0.002):
        target='buy'
    elif(value>=0.002):
        target='strong buy'
    elif(value<=-0.001 and value>=-0.002):
        target='sell'
    elif(value<-0.002):
        target='strong sell'
    return target


def classificateWithNumbers(value):
    target=0
    if(value>=0.001 and value<=0.002):
        target=1
    elif(value>=0.002):
        target=2
    elif(value<=-0.001 and value>=-0.002):
        target=-1
    elif(value<-0.002):
        target=-2
    return target

def classificateWithNumbersThreeTarget(value):
    target=0
    if(value>=0.0015):
        target=1
    elif(value<=-0.0015):
        target=-1
    else:
        target=0
    return target



def classificateWithIncreasingNumbers(value):
    target=3
    if(value>=0.0015 and value<=0.0025):
        target=4
    elif(value>=0.0025):
        target=5
    elif(value<=-0.0015 and value>=-0.0025):
        target=2
    elif(value<-0.0025):
        target=1
    return target

def correlation_heatmap(df):
    correlations = df.corr()

    fig, ax = plt.subplots(figsize=(40,40))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.1, annot=True, cbar_kws={"shrink": .7})
    sns.set(font_scale=2)
    plt.show();

def aboveValue(openList,inspectList,df,name):
    results=[]
    for i in range(len(inspectList)):
        if (openList[i]>inspectList[i]):
            results.append(1)
        else:
            results.append(0)
    df[name]=results
    

In [4]:
#data1 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_06', sep=",", header=None)
#data1.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

#data2 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_07', sep=",", header=None)
#data2.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

#data3 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_08', sep=",", header=None)
#data3.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data4 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_09', sep=",", header=None)
data4.columns = ["time", "open", "high", "low","close","volume"]

#data= pd.concat([data1,data2,data3,data4], axis=0)
#data

data=data4

In [5]:
%%time
#btc_data = pd.read_csv(r"/home/balazs/Desktop/szakdoga_info/btc_data.csv")
#data = btc_data.dropna()
data['MA5']=data['open'].rolling(window =5).mean()
data['EMA5']=data['open'].ewm(span=5,min_periods=0,adjust=False,ignore_na=False).mean()
data['CMA']=data.open.expanding().mean()
data['open_percentage_change']=data.open.pct_change()
data['close_percentage_change']=data.close.pct_change()
data['high_percentage_change']=data.high.pct_change()
data['low_percentage_change']=data.low.pct_change()
data['volume_percentage_change']=data.volume.pct_change()
lista=[]
for i in range(len(data.open_percentage_change.values)-5):
        value=data.open_percentage_change.values[i+1]+data.open_percentage_change.values[i+2]+data.open_percentage_change.values[i+3]+data.open_percentage_change.values[i+4]+data.open_percentage_change.values[i+5]
        lista.append(value)
for i in range(5):
    lista.append(0)
data['profit']=lista
buy_sell_list=[classificateWithNumbersThreeTarget(elem) for elem in data.profit]
data['decision']=buy_sell_list

CPU times: user 7.97 s, sys: 49.1 ms, total: 8.02 s
Wall time: 8.04 s


In [6]:
data=data.dropna()

In [7]:
data.drop(data.tail(5).index,inplace=True) # drop last n rows

In [8]:
aboveValue(data.open.values,data.MA5.values,data,'aboveMA5')
aboveValue(data.open.values,data.EMA5.values,data,'aboveEMA5')
aboveValue(data.open.values,data.CMA.values,data,'aboveCMA')
data

,time,open,high,low,close,volume,MA5,EMA5,CMA,open_percentage_change,close_percentage_change,high_percentage_change,low_percentage_change,volume_percentage_change,profit,decision,aboveMA5,aboveEMA5,aboveCMA
4,1595706420,9687.92,9692.32,9687.92,9691.73,0.035786,9688.828,9688.994568,9688.828000,-0.000369,-0.000061,0.000000,-0.000369,-0.478041,0.000959,0,0,0,0
5,1595706480,9691.73,9698.13,9691.73,9698.13,0.865957,9689.064,9689.906379,9689.311667,0.000393,0.000660,0.000599,0.000393,23.198503,-0.000637,0,1,1,1
6,1595706540,9696.89,9698.60,9691.31,9692.71,3.622457,9690.886,9692.234252,9690.394286,0.000532,-0.000559,0.000048,-0.000043,3.183185,-0.001400,0,1,1,1
7,1595706600,9698.68,9698.68,9698.68,9698.68,0.076229,9693.344,9694.382835,9691.430000,0.000185,0.000616,0.000008,0.000760,-0.978956,-0.001591,-1,1,1,1
8,1595706660,9693.49,9699.60,9693.49,9699.60,0.448836,9693.742,9694.085223,9691.658889,-0.000535,0.000095,0.000095,-0.000535,4.887971,-0.000947,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357363,1617147960,58631.74,58699.00,58631.74,58666.54,10.421432,58609.484,58615.089698,25394.927318,0.000542,0.001135,0.001689,0.000542,0.000889,0.000773,0,1,1,1
357364,1617148020,58680.18,58702.79,58680.18,58702.79,0.144199,58624.130,58636.786466,25395.021125,0.000826,0.000618,0.000065,0.000826,-0.986163,-0.000053,0,1,1,1
357365,1617148080,58699.45,58699.45,58675.12,58677.70,1.165074,58642.228,58657.674310,25395.114986,0.000328,-0.000427,-0.000057,-0.000086,7.079629,0.000328,0,1,1,1
357366,1617148140,58673.20,58673.20,58644.13,58644.13,0.843998,58656.912,58662.849540,25395.208772,-0.000447,-0.000572,-0.000447,-0.000528,-0.275584,0.001073,0,1,1,1


In [9]:
X=data[['close_percentage_change','high_percentage_change','low_percentage_change']]
y=data[['decision']]

In [14]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X, y = smote.fit_resample(X, y)

In [15]:
from imblearn.under_sampling import TomekLinks
TK = TomekLinks(sampling_strategy='majority')
X, y = TK.fit_resample(X, y)

In [16]:
y

,decision
0,0
1,0
2,0
3,-1
4,0
...,...
640721,1
640722,1
640723,1
640724,1


In [17]:
correlation_heatmap(data)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

# tanítás1

In [18]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [21]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [22]:
classes = ['-1','0','1']
print(classification_report(y_test, y_pred, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.66      0.69      0.67     60907
           0       0.58      0.54      0.56     65723
           1       0.65      0.67      0.66     65588

    accuracy                           0.63    192218
   macro avg       0.63      0.63      0.63    192218
weighted avg       0.63      0.63      0.63    192218



In [64]:
X_test.values

array([[-5.57828922e-04,  8.68393547e-04,  2.58810555e-03],
       [-3.46053729e-04,  3.49628666e-04,  2.22176048e-04],
       [ 1.11485143e-04, -1.47316155e-04, -8.03070943e-05],
       ...,
       [-1.96286472e-03, -2.54726117e-04, -1.76926061e-03],
       [ 5.50583636e-05,  1.36767194e-04,  2.14142672e-05],
       [-1.12255674e-03, -8.76902537e-04, -8.96679850e-04]])

In [62]:
clf.predict([[0.000587,0.000618,0.000963]])

array([0])

In [17]:
joslat=clf.predict([[-1.63950453e-04, -1.62325046e-04, -8.54107954e-04],
                    [-100.63950453e-04, -100.62325046e-04, -100.54107954e-04]
                   ])

In [18]:
joslat[1]

-1

In [19]:
import joblib
joblib.dump(clf,'decisionTree.joblib')

['decisionTree.joblib']

# tanítás2

In [20]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy',max_depth=10,min_samples_leaf=2,ccp_alpha=0.0001)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [21]:
classes = ['-1','0','1']
print(classification_report(y_test, y_pred, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.57      0.48      0.52     61206
           0       0.50      0.60      0.55     65566
           1       0.56      0.52      0.54     65435

    accuracy                           0.54    192207
   macro avg       0.54      0.54      0.54    192207
weighted avg       0.54      0.54      0.54    192207



# mindenféle tanítás

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0, stratify=y)

In [16]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

%matplotlib inline

In [17]:
%time
lr1 = LogisticRegression(random_state=0).fit(X_train, y_train)
print("train score - " + str(lr1.score(X_train, y_train)))
print("test score - " + str(lr1.score(X_test, y_test)))

train score - 0.4914175197100381
test score - 0.4901320309279638


In [18]:
%time
svc = LinearSVC(random_state=0).fit(X_train,y_train)
print("train score - " + str(svc.score(X_train, y_train)))
print("test score - " + str(svc.score(X_test, y_test)))

train score - 0.48542383608243345
test score - 0.4855540427445445


In [19]:
%time
forest = RandomForestClassifier(n_estimators=10, random_state=0).fit(X_train, y_train)
print("train score - " + str(forest.score(X_train, y_train)))
print("test score - " + str(forest.score(X_test, y_test)))

train score - 0.9872215247981113
test score - 0.6728455975117729


In [20]:
%time
gb = GradientBoostingClassifier().fit(X_train, y_train)
print("train score - " + str(gb.score(X_train, y_train)))
print("test score - " + str(gb.score(X_test, y_test)))

train score - 0.5317057328261915
test score - 0.5295789250159262


In [22]:
%time
knr = KNeighborsClassifier(n_neighbors=6).fit(X_train, y_train)
print("train score - " + str(knr.score(X_train, y_train)))
print("test score - " + str(knr.score(X_test, y_test)))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10 µs
train score - 0.7625260493110074
test score - 0.6592740172627003


In [31]:
%time
forest.predict([[0.001,0.003,0.003]])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


array([1])

In [32]:
%time
knr.predict([[0.001,0.1,0.0]])

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15.7 µs


array([1])

In [33]:

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)

>>> print(neigh.predict([[1.1]]))
[0]
>>> print(neigh.predict_proba([[0.9]]))
[[0.666... 0.333...]]

,close_percentage_change,high_percentage_change,low_percentage_change
0,-0.000061,0.000000,-0.000369
1,0.000660,0.000599,0.000393
2,-0.000559,0.000048,-0.000043
3,0.000616,0.000008,0.000760
4,0.000095,0.000095,-0.000535
...,...,...,...
640448,0.000061,0.000587,0.000265
640449,0.000018,0.000018,-0.000374
640450,0.000425,0.000973,0.001929
640451,0.000672,0.001340,-0.000233


In [35]:
from sklearn.neighbors import KNeighborsClassifier

In [36]:
model_kn = KNeighborsClassifier(n_neighbors=9, leaf_size=20)

In [37]:
model_kn.fit(X_train, y_train)

KNeighborsClassifier(leaf_size=20, n_neighbors=9)

In [38]:
pred_kn = model_kn.predict(X_test)

In [41]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(y_test, pred_kn, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.63      0.75      0.68     50562
           0       0.64      0.49      0.55     54776
           1       0.66      0.70      0.68     54776

    accuracy                           0.64    160114
   macro avg       0.64      0.64      0.64    160114
weighted avg       0.64      0.64      0.64    160114



# final tanitás

In [10]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
X, y = smt.fit_resample(X, y)

In [11]:
y.value_counts()

decision
-1          202530
 1          202303
 0          193353
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

def dtree_grid_search(X,y,nfolds):
    #create a dictionary of all values we want to test
    tree_param = {'max_depth':[None,4,5,6,7,8,9,10,11,12,15,20],
                 'ccp_alpha':[0.001,0.002,0.003,0.05,0.01,0.02]}
    # decision tree model
    dtree_model=DecisionTreeClassifier()
    #use gridsearch to test all values
    dtree_gscv = GridSearchCV(dtree_model, tree_param, cv=nfolds, verbose=1)
    #fit model to data
    dtree_gscv.fit(X, y)
    return dtree_gscv.best_params_

In [22]:
%%time
dtree_grid_search(X_train,y_train,4)

Fitting 4 folds for each of 72 candidates, totalling 288 fits
CPU times: user 21min 53s, sys: 1.1 s, total: 21min 54s
Wall time: 21min 58s


{'ccp_alpha': 0.001, 'max_depth': None}

In [26]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(max_depth=None,ccp_alpha=0.0001)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [27]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(y_test, y_pred, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.56      0.56      0.56     61001
           0       0.50      0.54      0.52     57747
           1       0.57      0.53      0.55     60708

    accuracy                           0.54    179456
   macro avg       0.54      0.54      0.54    179456
weighted avg       0.54      0.54      0.54    179456



In [28]:
import joblib
joblib.dump(clf,'decisionTree.joblib')

['decisionTree.joblib']